In [3]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# define geometry

# geo = SplineGeometry()
# get cavity geometry
# cav_geom = pd.read_csv('D:\Dropbox\multipacting\MPGUI21\mid_TESLA\geodata.n', 
#                        header=None, skiprows=3, skipfooter=1, sep='\s+', engine='python')[[1, 0]]
cav_geom = pd.read_csv('D:\Dropbox\CavityDesignHub\PhD_Thesis\SimulationData\Multipacting\C3795_n5\geodata.n', 
                       header=None, skiprows=3, skipfooter=1, sep='\s+', engine='python')[[1, 0]]
# cav_geom = pd.read_csv('D:\Dropbox\CavityDesignHub\PhD_Thesis\SimulationData\Multipacting\C3795_n5\geodata.n', 
#                        header=None, skiprows=3, skipfooter=1, sep='\s+', engine='python')[[1, 0]]

# # get underside of geometry
# cav_geom_flip = cav_geom.copy(deep=True)
# cav_geom_flip[0] = -1*cav_geom[0]
# cav_geom_flip = cav_geom_flip.iloc[::-1]

# # concatenate dataframes
# cav_geom = pd.concat([cav_geom, cav_geom_flip])

pnts = list(cav_geom.itertuples(index=False, name=None))

wp = WorkPlane()
wp.MoveTo(*pnts[0])
for p in pnts[1:]:
  wp.LineTo(*p)
wp.Close().Reverse()
face = wp.Face()
# Draw(face)
# Draw(face.Revolve(Axis((0,0,0), X), 360))

geo = OCCGeometry(face, dim=2)
help(geo)
print(len(face.edges))

Help on OCCGeometry in module netgen.libngpy._NgOCC object:

class OCCGeometry(netgen.libngpy._meshing.NetgenGeometry)
 |  Use LoadOCCGeometry to load the geometry from a *.step file.
 |  
 |  Method resolution order:
 |      OCCGeometry
 |      netgen.libngpy._meshing.NetgenGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Draw(...)
 |      Draw(self: netgen.libngpy._NgOCC.OCCGeometry) -> None
 |  
 |  GenerateMesh(...)
 |      GenerateMesh(self: netgen.libngpy._NgOCC.OCCGeometry, mp: netgen.libngpy._meshing.MeshingParameters = None, comm: netgen.libngpy._meshing.MPI_Comm = <netgen.libngpy._meshing.MPI_Comm object at 0x00000207EAC0E170>, mesh: netgen.libngpy._meshing.Mesh = None, **kwargs) -> netgen.libngpy._meshing.Mesh
 |      
 |      
 |      Meshing Parameters
 |      -------------------
 |      
 |      maxh: float = 1e10
 |        Global upper bound for mesh size.
 |      
 |      grading: float = 0.3
 |        Me

In [5]:
ngmesh = geo.GenerateMesh(maxh=0.05)

In [14]:
mesh = Mesh(ngmesh)
# help(Mesh.edges)
for ed in mesh.edges:
#     print(ed)
    pass
help(ed)
Draw (mesh);

Help on MeshNode in module ngsolve.comp object:

class MeshNode(NodeId)
 |  a node within a mesh
 |  
 |  Method resolution order:
 |      MeshNode
 |      NodeId
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  edges
 |      tuple of global edge numbers
 |  
 |  elements
 |      tuple of global element-ids
 |  
 |  faces
 |      tuple of global face numbers
 |  
 |  point
 |      vertex coordinates
 |  
 |  vertices
 |      tuple of global vertex numbers
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from NodeId:
 |  
 |  __eq__(...)
 |      __eq__(self: ngsolve.comp.NodeId, arg0: ngsolve.comp.NodeId) -> bool
 |  
 |  __hash__(...)
 |      __hash

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [7]:
fes = HCurl(mesh, order=3, dirichlet=[1, 2, 3, 4, 5, 6, 7, 8, 9])

print ("ndof =", fes.ndof, "FreeDofs", fes.FreeDofs())
mu0 = 4*pi*1e-7
c0 = 299792458
u,v = fes.TnT()

a = BilinearForm(y*curl(u)*curl(v)*dx)
m = BilinearForm(y*u*v*dx)


apre = BilinearForm(y*curl(u)*curl(v)*dx + y*u*v*dx)
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")

ndof = 106608 FreeDofs 0: 01000011011011011011011011111111111111111111111111
50: 11111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111111111111111111111
150: 11111111111111111111111111111111111111111111111111
200: 11111111111111111111111111111111111111111111111111
250: 11111111111111111111111111111111111111111111111111
300: 11111111111111111111111111111111111111111111111111
350: 11111111111111111111111111111111111111111111111111
400: 11111111111111111111111111111111111111111111111111
450: 11111111111111111111111111111111111111111111111111
500: 11111111111111111111111111111111111111111111111111
550: 11111111111111111111111111111111111111111111111111
600: 11111111111111111111111111111111111111111111111111
650: 11111111111111111111111111111111111111111111111111
700: 11111111111111111111111111111111111111111111111111
750: 11111111111111111111111111111111111111111111111111
800: 11111111111111111111111111111111111111111111111111
850: 11111111111111111111111

In [21]:
with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble()

    # build gradient matrix as sparse matrix (and corresponding scalar FESpace)
    gradmat, fesh1 = fes.CreateGradient()


    gradmattrans = gradmat.CreateTranspose() # transpose sparse matrix
    math1 = gradmattrans @ m.mat @ gradmat   # multiply matrices
    math1[0, 0] += 1     # fix the 1-dim kernel
    invh1 = math1.Inverse(inverse="sparsecholesky")

    # build the Poisson projector with operator Algebra:
    proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat

    projpre = proj @ pre.mat

    evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=projpre, num=5, maxit=20);

0 : [1121.4351289096592, 1835.4358321817438, 2126.5620875697005, 2777.6058406064803, 3389.5713019687364]
1 : [1054.2450137644562, 1101.7413273010643, 1148.7528717082855, 1271.8219159187918, 2190.264470482704]
2 : [1032.3081383606705, 1047.8097490873463, 1089.4017989454085, 1105.4956633277639, 1209.7047876702588]
3 : [1029.3804439362398, 1031.0173489989031, 1086.1561441815081, 1093.2952883703645, 1098.6314333109544]
4 : [1029.1847793347933, 1029.2882660943776, 1085.9052403484543, 1091.106812402114, 1095.9946525932771]
5 : [1029.171842065155, 1029.1800189103747, 1085.8858569546012, 1090.9156080717355, 1095.8552849363555]
6 : [1029.1708835902225, 1029.1724084538112, 1085.8841943091827, 1090.8989415890055, 1095.8445483832188]
7 : [1029.1707866560935, 1029.1718486146258, 1085.884040238639, 1090.8973732543898, 1095.8436459565678]
8 : [1029.1707785876843, 1029.1718036456243, 1085.8840251421989, 1090.8972176214934, 1095.8435661864885]
9 : [1029.1707779848432, 1029.1717998069341, 1085.884023598

In [22]:
print ("Eigenvalues")
for lam in evals:
    print (lam, 'freq: ', c0*np.sqrt(lam)/(2*np.pi) * 1e-6, "MHz")

Eigenvalues
1029.1707779346902 freq:  1530.680528978664 MHz
1029.1717994374326 freq:  1530.6812886164244 MHz
1085.8840234156423 freq:  1572.2896525287106 MHz
1090.8971997779513 freq:  1575.9148504027462 MHz
1095.84355810476 freq:  1579.483575041066 MHz


In [19]:
gfu = GridFunction(fes, multidim=len(evecs))
for i in range(len(evecs)):
    gfu.vecs[i].data = evecs[i]

Draw (gfu.MDComponent(4)[0], mesh, order=2);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [ ]:
# help(HCurl)
fess = H1(mesh, order=3)
u, v = fess.TnT()
c = grad(u)[0]

In [ ]:
# SetHeapSize(100*1000*1000)

fess = H1(mesh, order=3)

print ("ndof =", fess.ndof)
mu0 = 4*pi*1e-7
c0 = 299792458
u,v = fess.TnT()

a = BilinearForm(4/y**2*grad(u)[0]*grad(v)[0]*dx + grad(u)[1]*grad(v)[1]*dx)
m = BilinearForm(4/y**2*u*v*dx)

apre = BilinearForm(4/y**2*grad(u)[0]*grad(v)[0]*dx + grad(u)[1]*grad(v)[1]*dx + 4/y**2*u*v*dx)
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")

In [ ]:
with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble()

#     # build gradient matrix as sparse matrix (and corresponding scalar FESpace)
#     gradmat, fesh1 = fess.CreateGradient()

#     gradmattrans = gradmat.CreateTranspose() # transpose sparse matrix
#     math1 = gradmattrans @ m.mat @ gradmat   # multiply matrices
#     math1[0,0] += 1     # fix the 1-dim kernel
#     invh1 = math1.Inverse(inverse="sparsecholesky")

#     # build the Poisson projector with operator Algebra:
#     proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat

#     projpre = proj @ pre.mat

    evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=pre, num=200, maxit=20);

In [ ]:
print ("Eigenvalues")
for i, lam in enumerate(evals):
    print (i, lam, 'freq: ', c0*np.sqrt(lam)/(2*np.pi) * 1e-6, "MHz")

In [ ]:
gfu = GridFunction(fess, multidim=len(evecs))
for i in range(len(evecs)):
    gfu.vecs[i].data = evecs[i]

Draw (gfu.MDComponent(1)[0], mesh, order=2);

In [ ]:
help(OCCGeometry)